### Imports

In [ ]:
import re
import requests
import io
import string
from PyPDF2 import PdfFileReader

### Get PDF text

In [ ]:
pdfurl = "http://arxiv.org/pdf/1811.04422v1"
r = requests.get(pdfurl, stream=True)
f = io.BytesIO(r.content)
reader = PdfFileReader(f)

# Get text
text = ""
page = 0
while True:
    try:
        content = reader.getPage(page).extractText()
        text += content
        page += 1
    except:
        break

### Extract abstract

In [ ]:
text = text.split("\n\n")

# collect abstract text
abstract = []
cnt = 0

if "\n \n" in " ".join(i for i in text):
    content = []
    for i in text:
        for j in i.split("\n \n"):
            content.append(re.sub("\n", "", j))
    while cnt < len(content):
        if "abstract" in content[cnt].lower():
            if "abstract" in content[cnt].lower() and len(text[cnt].split()) < 10:
                cnt += 1
            while True:
                if "introduction" in content[cnt].lower():
                    break
                abstract.append(content[cnt])
                cnt += 1
            break
        else:
            cnt += 1
else:
    while cnt < len(text):
        if "abstract" in text[cnt].lower():
            if "abstract" in text[cnt].lower() and len(text[cnt].split()) < 10:
                cnt += 1
            while True:
                if "*" in text[cnt] and len(text[cnt].split()) < 10:
                    cnt += 1
                elif len(text[cnt].split()) > 10:
                    if "introduction" in text[cnt][:20].lower() or "i ntroduction" in text[cnt][:20].lower():
                        break
                    abstract.append(text[cnt])
                    cnt +=1
                elif len(text[cnt].split()) == 1:
                    try:
                        int(text[cnt])
                    except:
                        break
                    break
                else:
                    break
            break
        else:
            cnt += 1
    if not abstract:
        stop = 0
        while True:
            if "introduction" in text[stop].lower() and len(text[stop].split()) < 5:
                break
            else:
                stop += 1
        cnt = 0
        while cnt < stop:
            if text[cnt] == " " or len(text[cnt]) < 2:
                if len(text[cnt+1].split()) < 10:
                    cnt += 1
                else:
                    abstract.append(text[cnt+1])
                    if text[cnt+1][-1] == "-" or text[cnt+1][:-1] == "-":
                        while True:
                            if text[cnt+2][0].islower():
                                abstract.append(text[cnt+2])
                                break
                            else:
                                cnt += 1
            cnt += 1
            
# format abstract to present to user
abstract_string = ""
for a in abstract:
    a = a.strip()
    a = re.sub("-\n", "", a)
    a = re.sub("\n", " ", a)
    if a[-1] not in string.punctuation:
        abstract_string += a+" "
    elif a[-1] == "-" or a[-1] == "-":
        abstract_string += a[:-1]
    elif a[-1] in string.punctuation:
        abstract_string += a+" "
# remove word 'abstract' and date at start
if "abstract" in abstract_string[:8].lower():
    abstract_string = abstract_string[8:]
dummy1 = abstract_string.split(". ")
dummy2 = abstract_string.split()
if "abstract" in dummy1[0].lower():
    print(dummy1[0])
    cnt = 0
    while True:
        if dummy2[cnt].lower().strip() != "abstract":
            cnt += 1
        else:
            cnt += 1
            break
    abstract_string = " ".join(word for word in dummy2[cnt:])
if abstract_string[:1] in string.punctuation or abstract_string[:1] == "—":
    abstract_string = abstract_string[1:]
abstract_string = abstract_string.strip()
print("Abstract:\n\n", abstract_string)

### Testing the function

In [1]:
from abstractapi import abstractextracter
import gensim
from gensim import corpora

In [2]:
abstractextracter("http://arxiv.org/pdf/1811.04422v1")

'I describe an optimal control view of adversarial machine learning, where the dynamical system is the machine learner, the input are adversarial actions, and the control costs are defined by the adversary’s goals to do harm and be hard to detect. This view encompasses many types of adversarial machine learning, including test-item attacks, training-data poisoning, and adversarial reward shaping. The view encourages adversarial machine learning researcher to utilize advances in control theory and reinforcement learning.'

In [5]:
from PyPDF2 import PdfFileReader
from pathlib import Path
import requests

In [6]:
pdfurl = "http://arxiv.org/pdf/1811.04422v1"

url_name = pdfurl.split("/")[-1]
if ".pdf" not in url_name:
    pdf = Path(url_name+".pdf")
pdf = Path(url_name)
response = requests.get(pdfurl)
pdf.write_bytes(response.content)

137564

In [12]:
reader = PdfFileReader(url_name)
cnt = 0
text = ""
while True:
    content = reader.getPage(cnt).extractText()
    text += content
    if "abstract" in content.lower():
        break
    cnt += 1

In [15]:
import pdfplumber

In [17]:
pdf = pdfplumber.open("1811.04422v1")
page = pdf.pages[0]
text = page.extract_text()

In [18]:
page = pdf.pages[0]

In [19]:
text = page.extract_text()

In [22]:
text

'An Optimal Control View of Adversarial Machine Learning\nXiaojin Zhu\nDepartment of Computer Sciences, University of Wisconsin-Madison\n8\n1 Abstract\n0\nIdescribeanoptimalcontrolviewofadversarialmachinelearning,wherethedynamicalsystemisthe\n2\nmachine learner, the input are adversarial actions, and the control costs are deﬁned by the adversary’s\n \nv goals to do harm and be hard to detect. This view encompasses many types of adversarial machine\no learning,includingtest-itemattacks,training-datapoisoning, andadversarial rewardshaping. Theview\nN\nencouragesadversarialmachinelearningresearchertoutilizeadvancesincontroltheoryandreinforcement\n  learning.\n1\n1\n \n  1 Adversarial Machine Learning is not Machine Learning\n]\nG\nL Machine learning has its mathematical foundation in concentration inequalities. This is a consequence of\n. the independent and identically-distributed (i.i.d.) data assumption. In contrast, I suggest that adversarial\ns\nmachine learning may adopt optimal con

In [ ]:
stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [6]:
# Topic Modeling
final_topics = []
dictionary = corpora.Dictionary(abstract)

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [ ]:

        doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
        LDA = gensim.models.ldamodel.LdaModel
        LDA_fit = LDA(doc_term_matrix, num_topics=num_topics,
                        id2word=dictionary,
                        passes=200)
        #t0, t1, t2, t3, t4 = LDA_fit.print_topics(num_topics=-1, num_words=num_words)
        key_topics = []
        topics_list = [t for t in LDA_fit.print_topics(num_topics=-1, num_words=num_words)]
        for topic in topics_list:
            topics = topic[1]
            key_topics.append(topics)
        for i in key_topics:
            topic = i.replace('"', '')
            topic = topic.split('*')[1:]
            topic = [j.split(" + ")[0] for j in topic]
            new_topic = []
            for j in topic:
                #topic_ = j.decode('utf-8')
                if j == ' ':
                    pass
                elif j == '':
                    pass
                else:
                    new_topic.append(j.strip())
            final_topics.append(new_topic)
        if final_topics[0][0] == "":
            final_topics = None
        else:
            final_topics = [' + '.join(word for word in topic) for topic in final_topics]
    return final_topics